*Nombre de Grupos*: Grupo 1

*Integrantes*:

-Cevasco, Jorge

-Padilla, Ramiro

-Ranieri, Martina

## **Ejercicio**: Visualizar Difference of Gaussians (DoG)

Implementar una piramide Gaussiana (Aplicar un filtro Gaussiano consecutivamente a una imagen) y calcular Difference of Gaussians (DoG) para una imagen de entrada. Visualizar el resultado.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, filters, data
from scipy import ndimage

# Cargamos una imagen y la pasamos a gris.
image = data.camera()
#gray = color.rgb2gray(image)

# Definimos la escalas de sigmas que usaremos en la piramide.
sigmas = [1.0, 2.0, 4.0, 8.0]
gaussians = []

# Generamos las imagenes blureadas
for sigma in sigmas:
    imagen_blur = filters.gaussian(image, sigma)
    gaussians.append(imagen_blur)

# Calculamos DoG (Difference of Gaussians)
diferencias = [1, -1]
dogs = []
for i in range(len(gaussians)-1):
  dogs.append(gaussians[i+1]-gaussians[i])

for dog in dogs:
  n,m = dog.shape
  for i in range(1,n-1):
    for j in range(1,m-1):
      cuadro = dog[i-1:i+2, j-1:j+2]
      if ((dog[i,j] > cuadro.min()) and (dog[i,j] < cuadro.max())):
        dog[i,j] = 0


for dog in dogs:
  dog[np.nonzero(dog)] = 255


#np.nonzero(dogs)

fig, axes = plt.subplots(1, len(dogs), figsize=(15, 5))
for i, ax in enumerate(axes):
    ax.imshow(dogs[i], cmap='gray')
    ax.set_title(f'DoG {i+1}: σ={sigmas[i+1]} - σ={sigmas[i]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

## **Muestra**: Deteccion de Keypoint y Matcheo

Detectar keypoints de SIFT en dos imagenes y matchear los descriptores.Visualizamos el matcheo.


In [ ]:
from skimage import data, color, feature, transform
from skimage.feature import match_descriptors, plot_matched_features
import matplotlib.pyplot as plt
import numpy as np

# Cargamos una imagen de ejemplo
image1 = color.rgb2gray(data.astronaut())

# Creamos una copia transformada para poder matchear
tform = transform.AffineTransform(scale=(0.8, 0.8), rotation=np.deg2rad(25), translation=(50, -30))
image2 = transform.warp(image1, tform.inverse)

# Inicializamos Sift y encontramos los keypoints
sift1 = feature.SIFT()
sift1.detect_and_extract(image1)

sift2 = feature.SIFT()
sift2.detect_and_extract(image2)

# Macheamos los keypoints entre las dos imagenes
matches = match_descriptors(sift1.descriptors, sift2.descriptors, cross_check=True)

fig, ax = plt.subplots(figsize=(12, 6))

plot_matched_features(
    image1,
    image2,
    keypoints0=sift1.keypoints,
    keypoints1=sift2.keypoints,
    matches=matches[::15], # Mostramos solo algunos matches para que sea facil de ver.
    ax=ax)

ax.axis('off')
ax.set_title('SIFT Matcheo de Keypoints (Original vs Transformada)')
plt.show()